## Notebook to create brain state files from EF1_ALPHA excel sleep scoring spreadsheet

In [3]:
import os 
import numpy as np
import pandas as pd

In [64]:
os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA')
epoch_number = "A, E, I, M, Q, U, Y, AC, AG, AK, AO, AS, BA, BE, BI, BM, BQ, BU, BY, CC, CG, CK, CO, CS, CW"
cols = "B, F, J, N, R, V, Z, AD, AH, AL, AP, AT, BB, BF, BJ, BN, BR, BV, BZ,CD, CH, CL, CP, CT, CX"
cols_discard = "C, G, K, O, S, W, AA, AE, AI, AM, AQ, AU, BC, BG, BK, BO, BS, BW, CA, CE, CI, CM, CQ, CU, CY"
skiprows = 5
raw_excel = pd.read_excel('grant_EEG_classification.xlsx', usecols = cols, header= 1)

In [65]:
#list of animal ids
column_names = list(raw_excel)
animal_ids = [anim_id[0:10] for anim_id in column_names]

In [74]:
#strip whitespace
animal_ids = [id.replace(" ", "") for id in animal_ids]
print(animal_ids)

['210422_D', '210423_D', '210705_C', '210706_C', '191125_A', '191126_A', '210705_B', '210706_B', '210705_A', '210706_A', '191216_A_1', '191217_A_1', '191104_B', '191107_A', '191108_A', '210422_B_1', '210423_B_1', '191216_C', '191217_C', '191216_B', '191217_B', '210429_C', '210430_C', '210705_D', '210706_D']


In [67]:
raw_epochs = pd.read_excel('grant_EEG_classification.xlsx', usecols = epoch_number, header = 1, skiprows = 6)
raw_epochs_letter = pd.read_excel('grant_EEG_classification.xlsx', usecols = cols, header = 1, skiprows = 6)
raw_discard = pd.read_excel('grant_EEG_classification.xlsx', usecols = cols_discard, header = 1, skiprows = 6)

In [68]:
two_files = []
for epoch in raw_epochs:
    start_epochs = raw_epochs[raw_epochs[epoch] == 0].index.tolist()
    two_files.append(start_epochs)

In [76]:
for file, epoch, animal_id, epoch_letter_col, discard_col in zip(two_files, raw_epochs, animal_ids, raw_epochs_letter, raw_discard):
    print(animal_id)
    if len(file) == 2:
        start_2 = file[1:2]
        start_2 = start_2[0]
        end_epoch = start_2 - 1
        epochs_letter = raw_epochs_letter[epoch_letter_col]
        drop_na = epochs_letter.dropna()
        length_col = len(drop_na) 
        epoch_column_raw = raw_epochs[epoch]
        part_1 = epoch_column_raw[0: end_epoch].astype(int)
        part_2 = epoch_column_raw[start_2: length_col].astype(int)
        epochs_letter_part_1 = epochs_letter[0:end_epoch]
        epochs_letter_part_2 = epochs_letter[start_2: length_col]
        discard_data = raw_discard[discard_col]
        discard_part_1 = discard_data[0: end_epoch]
        discard_part_2 = discard_data[start_2: length_col]
        part_1_dict = {'epoch_numbers': part_1, 'brain_state': epochs_letter_part_1,
                       'epoch_discard_numbers': discard_part_1}
        part_2_dict = {'epoch_numbers': part_2, 'brain_state': epochs_letter_part_2,
                      'epoch_discard_numbers': discard_part_2}
        df_1 = pd.DataFrame(data = part_1_dict)
        df_2 = pd.DataFrame(data = part_2_dict)
        os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
        df_1.to_csv(str(animal_id) +'_part_1.csv', index=True) 
        df_2.to_csv(str(animal_id) +'_part_2.csv', index=True)
    else:
        epochs_letter = raw_epochs_letter[epoch_letter_col]
        drop_na = epochs_letter.dropna()
        length_col = len(drop_na) 
        epoch_column_raw = raw_epochs[epoch]
        all_epochs = epoch_column_raw[0:length_col].astype(int)
        all_epochs_letter = epochs_letter[0:length_col]
        discard_data = raw_discard[discard_col]
        all_discard_epochs = discard_data[0:length_col] 
        data_dict = {'epoch_numbers': all_epochs, 'brain_state': all_epochs_letter,
                       'epoch_discard_numbers': all_discard_epochs}
        os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
        data_df = pd.DataFrame(data = data_dict)
        data_df.to_csv(str(animal_id) +'.csv', index=True)

210422_D
210423_D
210705_C
210706_C
191125_A
191126_A
210705_B
210706_B
210705_A
210706_A
191216_A_1
191217_A_1
191104_B
191107_A
191108_A
210422_B_1
210423_B_1
191216_C
191217_C
191216_B
191217_B
210429_C
210430_C
210705_D
210706_D


In [ ]:
os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
test_check = pd.read_csv('191125_A_part_2.csv')

In [ ]:
wake_indices = test_check.loc[test_check['brain_state'] == 'W'].index.tolist()
rem_indices = test_check.loc[test_check['brain_state'] == 'R']
nonrem_indices = test_check.loc[test_check['brain_state'] == 'N']
discard_indices = test_check.loc[test_check['epoch_discard_numbers'] == 'E']

In [ ]:
wake_indices_list = wake_indices.index.tolist()
rem_indices_list = rem_indices.index.tolist()
nonrem_indices_list = nonrem_indices.index.tolist()
discard_indices_list = discard_indices.index.tolist()

In [ ]:
def non_match_elements(list_a, list_b):
    non_match = []
    for i in list_a:
        if i not in list_b:
            non_match.append(i)
    return non_match

new_list = non_match_elements(wake_indices_list, discard_indices_list)

## check that code works on individual animals

In [4]:
os.chdir('/home/melissa/PROJECT_DIRECTORIES/ef1_alpha_analysis')
%run preprocess.py

In [5]:
#paths 
recording_path = '/home/melissa/PREPROCESSING/EF1_ALPHA'
brain_state_path = '/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder'

In [14]:
os.chdir('/home/melissa/PROJECT_DIRECTORIES/ef1_alpha_analysis')
%run ef1_alpha_properties.py
%run preprocess.py
%run filter.py
%run power.py

In [15]:
recordings_2_channels

{'A': ['191125', '191126'],
 'A_1': ['191217'],
 'B': ['191217', '210705'],
 'B_1': ['210422', '210423'],
 'C': ['210705', '191217'],
 'D': ['210422', '210423', '210705']}

In [16]:
channels_dict

{'A': [2, 3, 7, 9],
 'A_1': [2, 3, 7, 8],
 'B': [18, 19, 23, 25],
 'B_1': [18, 20, 23, 25],
 'C': [34, 35, 39, 41],
 'D': [50, 51, 55, 57]}

In [17]:
#2 lists of letters - 1 for one recording and 1 for two recording
recordings_letters_1 = ['A', 'A_1', 'B', 'C', 'D']
recording_letters_2 = ['A', 'A_1', 'B', 'B_1', 'C', 'D']

In [18]:
 print (dir(ExtractBrainStateEF1ALPHA)) 

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'create_epoch_bins', 'epoch_length', 'get_epoch_indices', 'load_brain_state_file', 'load_npy_recordings', 'remove_E_epochs', 'sample_rate']


In [19]:
os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/two_recording/Part_2')
part_2 = np.load('191217_part2_A_1.npy')

In [20]:
animal_id = '191125'
letter = 'A'
os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
state_file_names = os.listdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
for file in state_file_names:
    if file.startswith(animal_id) and file.endswith(letter + '_part_1.csv'):
        part_1_brain_state = pd.read_csv(file)
    if file.startswith(animal_id) and file.endswith(letter + '_part_2.csv'):
        part_2_brain_state = pd.read_csv(file)
        

In [ ]:
for brainstate in brainstates:
    for headstage_letters in recording_letters_2:
        print(headstage_letters)
        for anim_id in recordings_2_channels[headstage_letters]:
            print(anim_id)
            preprocessing_steps = ExtractBrainStateEF1ALPHA(anim_id, recording_path, brain_state_path, 
                                                            headstage_letters, recording_number=2)
            part_1, part_2 = preprocessing_steps.load_npy_recordings()
            part_1_br_state, part_2_br_state = preprocessing_steps.load_brain_state_file()
            if part_1 and part_2 is not None:
                epoch_indices_1 = preprocessing_steps.remove_E_epochs(part_1_br_state, brain_state_letter=brainstate)
                epoch_indices_2 = preprocessing_steps.remove_E_epochs(part_2_br_state, brain_state_letter=brainstate)
                epoch_bins_1 = preprocessing_steps.get_epoch_indices(part_1_br_state, epoch_indices_1)
                epoch_bins_2 = preprocessing_steps.get_epoch_indices(part_2_br_state, epoch_indices_2)
                for column, channel in zip(part_1_br_state.T, channels_dict[headstage_letters]):
                    filter_steps = Filter(column, epoch_bins_1)
                    filtered_data_1 = filter_steps.butter_bandpass()
                    power_steps = PowerSpectrum(filtered_data_1)
                    power_array_part_1, frequency_array_part_1 = power_steps.average_psd()
                for column, channel in zip(part_2_br_state.T, channels_dict[headstage_letters]):
                    filter_steps = Filter(column, epoch_bins_2)
                    filtered_data_2 = filter_steps.butter_bandpass()
                    power_steps = PowerSpectrum(filtered_data_2)
                    power_array_part_2, frequency_array_part_2 = power_steps.average_psd()
                average_power_array = average_power_df(power_array_part_1, power_array_part_2)
                dict_data = {'Animal_ID': [anim_id]*len(average_power_array), 'Headstage':[headstage_letters]*len(average_power_array),
                            'Channel': [channel]*len(average_power_array), 'Brainstate': [brainstate]*len(average_power_array),
                            'Power': average_power_array, 'Frequency': frequency_array_part_1,}
                two_recording.append(pd.DataFrame(data=dict_data))
    merged_power_file = pd.conctenate(one_recording, axis = 0).drop_duplicates().reset_index(drop = True)
    os.chdir('/home/melissa/RESULTS/EF1_ALPHA/2_REC')
    merged_power_file.to_csv(str(brainstate) + '_2_rec.csv', index = True)

In [ ]:
one_recording = []
for letters_1 in recordings_letters_1:
    for anim_id in recordings_1_channels[letters_1]:
        testing_class = ExtractBrainStateEF1ALPHA(anim_id, recording_path, brain_state_path, letters_1, recording_number = 1)
        npy_recording = testing_class.load_npy_recordings()
        print(npy_recording)
        br_state_files = testing_class.load_brain_state_file()
        if br_state_files is not None:
            br_state_indices = testing_class.remove_E_epochs(br_state_files, brain_state_letter = 'W')
            epoch_indices = testing_class.get_epoch_indices(br_state_indices)
            epoch_bins = testing_class.create_epoch_bins(br_state_files, epoch_indices)
            if npy_recording is not None:
                for column, channel in zip(npy_recording.T, channels_dict[letters_1]):
                    filter_test = Filter(column, epoch_bins)
                    filtered_data = filter_test.butter_bandpass()
                    power_test = PowerSpectrum(filtered_data)
                    power_array, frequency_array = power_test.average_psd()
                    dict_data = {'Animal_ID': [anim_id]*len(power_array), 'Headstage': [letters_1]*len(power_array),
                            'Channel': [channel]*len(power_array), 'Power': power_array, 'Frequency': frequency_array,
                                'BrainState': ['W']*len(power_array)}
                    one_recording.append(pd.DataFrame(data = dict_data))

In [ ]:
merged_power_file = pd.concat(one_recording, axis=0).drop_duplicates().reset_index(drop=True)

In [ ]:
os.chdir('/home/melissa/RESULTS/EF1_ALPHA/2_REC')
#merged_power_file.to_csv('one_recording_wake.csv', index=True)